<a href="https://colab.research.google.com/github/AudreyCTang/Movie-Review-Topic-Modelling-Sentiment-Analysis/blob/master/NLP_Movie_Description_(BERT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# verify GPU availability
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [4]:
from google.colab import files
uploaded = files.upload()

Saving IMDB_Reviews.csv to IMDB_Reviews.csv


In [5]:
import io
df = pd.read_csv(io.BytesIO(uploaded['IMDB_Reviews.csv']))
# Dataset is now stored in a Pandas Dataframe

In [ ]:
df.head()

,Unnamed: 0,Review,Sentiment
0,0,"Based on an actual story, John Boorman shows t...",1.0
1,1,This is a gem. As a Film Four production - the...,1.0
2,2,"I really like this show. It has drama, romance...",1.0
3,3,This is the best 3-D experience Disney has at ...,1.0
4,4,"Of the Korean movies I've seen, only three had...",1.0


In [ ]:
df=df.drop(columns='Unnamed: 0')

In [ ]:
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 25,000



,Review,Sentiment
18793,"Jack Frost 2. THE worst ""horror film"" I have e...",0.0
14801,"Mansfield Park, in its second half, is my favo...",0.0
19194,There was a time when the Alien series was a s...,0.0
15050,Oddball black-comedy romance featuring a great...,0.0
5316,One of the best films I've seen in a long time...,1.0
17888,"And also a wonderful beginning, a real quick s...",0.0
1771,Fuckland is an interesting film. I personally ...,1.0
11215,"I'm not a fan of scratching, but I really dug ...",1.0
15662,"I loved so much last Bellocchio's movie, the m...",0.0
11185,"Recently released on British DVD, this is a go...",1.0


# Topic Modelling
For the first part, we will use topic modelling techniques to group different reviews.

In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 5.2MB/s 
     |████████████████████████████████| 552kB 37.4MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=98fd7c9a4e2d491f36d4554010715e862acd0dd13cc87750290ad8836745eebf
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=31006a3565880ebae2f95c94931921298ec9f1828ff3183a7a5191913fc17ba6
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [ ]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import LsiModel, TfidfModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

from gensim.parsing.preprocessing import preprocess_string, remove_stopwords

import nltk
from nltk import word_tokenize, corpus
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import WordPunctTokenizer

from nltk.stem import RegexpStemmer, WordNetLemmatizer
from nltk.stem.porter import *

from nltk.corpus import stopwords

# Run in python console
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib.colors as mcolors
import matplotlib.style as style

from wordcloud import WordCloud, STOPWORDS

from IPython.display import display

# Get topic weights and dominant topics ------------
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook



# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
data = df.Review.values.tolist()

In [ ]:
labels = df.Sentiment.values

[1. 1. 1. ... 0. 0. 0.]


In [ ]:

def clean_text(x):
    
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', x)


    
    return x
    

    
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)

    return x

def clean(x):
    x = clean_text(x)
    x = clean_numbers(x)

    return x

def remove_stopwords(x):
    stopwords = ['movi', 'film', 'like', 'scene', 'watch']
    x  = [word for word in x if word.lower() not in stopwords]

    return x

def prepare_documents(documents):
    print('Preparing documents')
    documents = [clean(document) for document in documents]
    documents = [preprocess_string(doc) for doc in documents]
    documents = [remove_stopwords(doc) for doc in documents]

    return documents

def create_lda_model(documents, dictionary, number_of_topics):

    print(f'Creating LDA Model with {number_of_topics} topics')

    # using bow to construct relevance table out of documents
    document_terms = [dictionary.doc2bow(doc) for doc in documents]

    return LdaModel(document_terms,
                    num_topics=number_of_topics,
                    id2word = dictionary, passes=15), document_terms

def run_lda_process(documents, number_of_topics):

    documents = prepare_documents(documents)
    
    # mapping between words and their integer ids
    dictionary = corpora.Dictionary(documents)

    lda_model, document_terms = create_lda_model(documents, dictionary,
                                 number_of_topics)

    return documents, dictionary, lda_model, document_terms


In [ ]:
def calculate_coherence_score(documents, dictionary, model):

    coherence_model = CoherenceModel(model=model,
                                     texts=documents,
                                     dictionary=dictionary,
                                     coherence='c_v')
    return coherence_model.get_coherence()

def get_coherence_values(start, stop, document):

    for num_topics in range(start, stop):
        print(f'\nCalculating coherence for {num_topics} topics')
        documents, dictionary, model, document_terms = run_lda_process(document,
                                                       number_of_topics=num_topics)
        coherence = calculate_coherence_score(documents,
                                              dictionary,
                                              model)

        yield coherence

In [ ]:
min_topics, max_topics = 6,20

coherence_scores = list(get_coherence_values(min_topics, max_topics, data))


Calculating coherence for 6 topics
Preparing documents


KeyboardInterrupt: ignored

In [ ]:
style.use('fivethirtyeight')

x = [int(i) for i in range(min_topics, max_topics)]

plt.figure(figsize=(10,8))

plt.plot(x, coherence_scores)

plt.xlabel('Number of topics')

plt.ylabel('Coherence Value')

plt.title('Coherence Scores by number of Topics')

In [ ]:
NUM_TOPICS = 15

documents, dictionary, model, document_terms = run_lda_process(data, number_of_topics=NUM_TOPICS)


Preparing documents
Creating LDA Model with 15 topics


In [ ]:
# find dominant topics in each document

def format_topics_sentences(ldamodel=None, corpus=document_terms, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
#         print(row_list)
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
#         print(row)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
#             print(j, (topic_num, prop_topic))
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=model, corpus=document_terms, texts=documents)

# apply the above function to find dominant topics in data
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
# get 10 random samples
df_dominant_topic.sample(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
3442,3442,7.0,0.5111,"love, plai, girl, role, great, good, year, com...","[cherri, tell, naiv, unmarri, virgin, decid, b..."
20717,20717,4.0,0.5832,"bad, good, time, think, act, peopl, thing, loo...","[nightmar, sensat, feel, wake, nightmar, got, ..."
694,694,11.0,0.4335,"mummi, ranger, gai, gold, heist, mexican, east...","[year, prison, toni, stéphanoi, jean, servai, ..."
12163,12163,7.0,0.4267,"love, plai, girl, role, great, good, year, com...","[extrem, delight, bias, suppos, happen, ador, ..."
3506,3506,4.0,0.3495,"bad, good, time, think, act, peopl, thing, loo...","[year, old, releas, year, later, qualm, hire, ..."
6488,6488,4.0,0.9067,"bad, good, time, think, act, peopl, thing, loo...","[turn, better, expect, part, pretti, funni, ni..."
2088,2088,8.0,0.4694,"life, charact, peopl, stori, live, man, world,...","[lot, dialogu, flat, goofi, add, machin, daili..."
10109,10109,7.0,0.2565,"love, plai, girl, role, great, good, year, com...","[great, new, avail, dvd, http, treasureflix, c..."
1113,1113,6.0,0.3132,"charact, stori, good, time, great, actor, act,...","[battleship, potemkin, said, favourit, charli,..."
2326,2326,6.0,0.3829,"charact, stori, good, time, great, actor, act,...","[despit, titl, unlik, stori, love, war, isn, s..."


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, document_terms, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.287906  0.008558       1        1  21.181749
6     -0.259813 -0.017220       2        1  17.099566
8     -0.236518  0.028064       3        1  14.942209
14    -0.198092  0.060530       4        1  13.444116
0     -0.176628  0.068556       5        1   8.870417
7     -0.135626 -0.101511       6        1   8.538270
9      0.066336 -0.091754       7        1   2.613551
13     0.075147  0.131354       8        1   2.552893
3      0.044115 -0.127253       9        1   2.276260
2      0.139844 -0.157211      10        1   1.680173
12     0.178628  0.003357      11        1   1.441548
1      0.187819  0.226976      12        1   1.408853
11     0.199203 -0.075226      13        1   1.379766
10     0.193920 -0.099932      14        1   1.289342
5      0.209570  0.142711      15        1   1.281285, topic_info=         Term          Freq         Total Category  logprob  loglift
189     music   3895.000000   3895.000000  Default  30.0000  30.0000
87        bad  10253.000000  10253.000000  Default  29.0000  29.0000
446      plai   8340.000000   8340.000000  Default  28.0000  28.0000
422      love   7775.000000   7775.000000  Default  27.0000  27.0000
1039      war   2239.000000   2239.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
766    return    136.585419   1094.003052  Topic15  -5.3996   2.2767
2831    blade    100.161598    193.694427  Topic15  -5.7098   3.6978
2462    comic    119.516289   1036.601318  Topic15  -5.5331   2.1971
4011  cartoon    114.326988    676.499573  Topic15  -5.5775   2.5794
411      hero    100.433182   1337.465210  Topic15  -5.7071   1.7683

[1022 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
8396      10  0.993657   aaron
6119      12  0.979519  abbott
2654       3  0.919438    abus
2654       4  0.062293    abus
2654       6  0.017442    abus
...      ...       ...     ...
1292      10  0.010604   young
1292      11  0.000624   young
12079      2  0.990385    zeta
38038      1  0.992930    zoei
2544       4  0.998681   zombi

[2828 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 7, 9, 15, 1, 8, 10, 14, 4, 3, 13, 2, 12, 11, 6])

# Sentiment Analysis with BERT
In the second part, we will use sentiment analysis with BERT on the reviews according to topics.

In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 5.3MB/s 
     |████████████████████████████████| 1.1MB 30.3MB/s 
     |████████████████████████████████| 890kB 25.5MB/s 
     |████████████████████████████████| 3.0MB 56.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=37f61cbe35aa8759cbc90a59c74d9b6382760e45903ebad6395b1cc6208649ce
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [ ]:
df['Dominant_Topic']=df_dominant_topic['Dominant_Topic']


In [ ]:
labels=[]
documents=[]

groups=df.groupby(['Dominant_Topic'])
for name, sentiment in groups['Sentiment']:
  labels.append(sentiment.tolist())
for name, review in groups['Review']:
  documents.append(review.tolist())



In [ ]:
# Print the original sentence.
print(' Original: ', data[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(data[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(data[0])))

 Original:  Based on an actual story, John Boorman shows the struggle of an American doctor, whose husband and son were murdered and she was continually plagued with her loss. A holiday to Burma with her sister seemed like a good idea to get away from it all, but when her passport was stolen in Rangoon, she could not leave the country with her sister, and was forced to stay back until she could get I.D. papers from the American embassy. To fill in a day before she could fly out, she took a trip into the countryside with a tour guide. "I tried finding something in those stone statues, but nothing stirred in me. I was stone myself." <br /><br />Suddenly all hell broke loose and she was caught in a political revolt. Just when it looked like she had escaped and safely boarded a train, she saw her tour guide get beaten and shot. In a split second she decided to jump from the moving train and try to rescue him, with no thought of herself. Continually her life was in danger. <br /><br />Here 

In [ ]:
max_len = 0

# For every sentence...
for document in documents:
  for sentence in document:

      # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
      input_ids = tokenizer.encode(sentence, add_special_tokens=True)

      # Update the maximum sentence length.
      max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  3157


In [ ]:
# Tokenize all of the data and map the tokens to thier word IDs.
input_ids = [[] for x in range(NUM_TOPICS)]
attention_masks = [[] for x in range(NUM_TOPICS)]

# For every group...
topic_counter=0
for document in documents:
  # For every document...
  for sentence in document:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.
      encoded_dict = tokenizer.encode_plus(
                          sentence,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 3200,           # Pad & truncate all data.
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      
      # Add the encoded sentence to the list.    
      input_ids[topic_counter].append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks[topic_counter].append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
  input_ids[topic_counter] = torch.cat(input_ids[topic_counter], dim=0)
  attention_masks[topic_counter] = torch.cat(attention_masks[topic_counter], dim=0)
  labels[topic_counter] = torch.tensor(labels[topic_counter])

  # Print sentence 0, now as a list of IDs.
  print('Original: ', documents[topic_counter][0])
  print(topic_counter)
  print('Token IDs:', input_ids[topic_counter][0])
  topic_counter+=1


Original:  I played Sam (the porter, Lou's sidekick) in the Film "Dead Rail" Which later aired as "Alien Express." And, I have to say that for my part I thoroughly enjoyed watching this film. As a struggling actor this was a chance for me to work with fantastic people, it gave me great scenes to include on my reel, and it allowed me to work on a dream job for a month and a half (no waiting tables!) Turi(the director) And Steve and Scott (the producers) Were very kind by giving me this opportunity to participate in the production. I made many friends (Lou, Todd, Steven) and I consider myself very fortunate to have been able to work with these incredibly talented people. There was not a day that went by that I did not laugh my butt off. The real tragedy isn't so much the special effects, it's that every single person who watched this film didn't get to see what happened behind the scenes and all the talent that truly went into it. Craftsmen building the set, prop masters, gaffers,wardrob

In [ ]:
from torch.utils.data import TensorDataset, random_split
dataset= [[] for x in range(NUM_TOPICS)]
train_dataset= [[] for x in range(NUM_TOPICS)]
val_dataset= [[] for x in range(NUM_TOPICS)]
# Combine the training inputs into a TensorDataset.
for i in range(NUM_TOPICS):
  dataset[i] = TensorDataset(input_ids[i], attention_masks[i], labels[i])
  # Create a 90-10 train-validation split.

  # Calculate the number of samples to include in each set.
  train_size = int(0.9 * len(dataset[i]))
  val_size = len(dataset[i]) - train_size

  # Divide the dataset by randomly selecting samples.
  train_dataset[i], val_dataset[i] = random_split(dataset[i], [train_size, val_size])

  print('{:>5,} training samples'.format(train_size))
  print('{:>5,} validation samples'.format(val_size))

1,134 training samples
  126 validation samples
   52 training samples
    6 validation samples
   68 training samples
    8 validation samples
  137 training samples
   16 validation samples
7,930 training samples
  882 validation samples
   49 training samples
    6 validation samples
5,313 training samples
  591 validation samples
1,611 training samples
  180 validation samples
3,063 training samples
  341 validation samples
  207 training samples
   24 validation samples
   47 training samples
    6 validation samples
   37 training samples
    5 validation samples
   77 training samples
    9 validation samples
  175 training samples
   20 validation samples
2,592 training samples
  288 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

train_dataloader= [[] for x in range(NUM_TOPICS)]
validation_dataloader= [[] for x in range(NUM_TOPICS)]

for i in range(NUM_TOPICS):
# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 

  train_dataloader[i] = DataLoader(
              train_dataset[i],  # The training samples.
              sampler = RandomSampler(train_dataset[i]), # Select batches randomly
              batch_size = batch_size # Trains with this batch size.
          )

  # For validation the order doesn't matter, so we'll just read them sequentially.
  validation_dataloader[i] = DataLoader(
              val_dataset[i], # The validation samples.
              sampler = SequentialSampler(val_dataset[i]), # Pull out batches sequentially.
              batch_size = batch_size # Evaluate with this batch size.
          )

In [ ]:
print(train_dataloader[0])

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4
total_steps= []
scheduler= []

for i in range(NUM_TOPICS):
  # Total number of training steps is [number of batches] x [number of epochs]. 
  # (Note that this is not the same as the number of training samples).
  total_steps.append(len(train_dataloader[i]) * epochs)

  # Create the learning rate scheduler.
  scheduler.append(get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps = 0, # Default value in run_glue.py
                                              num_training_steps = total_steps[i]))


In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = [[] for x in range(NUM_TOPICS)]

# Measure the total training time for the whole run.
total_t0 = time.time()

for i in range(NUM_TOPICS):
  # For each epoch...
  for epoch_i in range(0, epochs):
      
      # ========================================
      #               Training
      # ========================================
      
      # Perform one full pass over the training set.

      print("")
      print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
      print('Training...')

      # Measure how long the training epoch takes.
      t0 = time.time()

      # Reset the total loss for this epoch.
      total_train_loss = 0

      # Put the model into training mode. Don't be mislead--the call to 
      # `train` just changes the *mode*, it doesn't *perform* the training.
      # `dropout` and `batchnorm` layers behave differently during training
      # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
      model.train()

      # For each batch of training data...
      for step, batch in enumerate(train_dataloader[i]):
          # Progress update every 40 batches.
          if step % 40 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)
              
              # Report progress.
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader[i]), elapsed))

          # Unpack this training batch from our dataloader. 
          #
          # As we unpack the batch, we'll also copy each tensor to the GPU using the 
          # `to` method.
          #
          # `batch` contains three pytorch tensors:
          #   [0]: input ids 
          #   [1]: attention masks
          #   [2]: labels 
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)

          # Always clear any previously calculated gradients before performing a
          # backward pass. PyTorch doesn't do this automatically because 
          # accumulating the gradients is "convenient while training RNNs". 
          # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
          model.zero_grad()        

          # Perform a forward pass (evaluate the model on this training batch).
          # The documentation for this `model` function is here: 
          # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
          # It returns different numbers of parameters depending on what arguments
          # arge given and what flags are set. For our useage here, it returns
          # the loss (because we provided labels) and the "logits"--the model
          # outputs prior to activation.
          loss, logits = model(b_input_ids, 
                              token_type_ids=None, 
                              attention_mask=b_input_mask, 
                              labels=b_labels)

          # Accumulate the training loss over all of the batches so that we can
          # calculate the average loss at the end. `loss` is a Tensor containing a
          # single value; the `.item()` function just returns the Python value 
          # from the tensor.
          total_train_loss += loss.item()

          # Perform a backward pass to calculate the gradients.
          loss.backward()

          # Clip the norm of the gradients to 1.0.
          # This is to help prevent the "exploding gradients" problem.
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          # Update parameters and take a step using the computed gradient.
          # The optimizer dictates the "update rule"--how the parameters are
          # modified based on their gradients, the learning rate, etc.
          optimizer.step()

          # Update the learning rate.
          scheduler.step()

      # Calculate the average loss over all of the batches.
      avg_train_loss = total_train_loss / len(train_dataloader[i])            
      
      # Measure how long this epoch took.
      training_time = format_time(time.time() - t0)

      print("")
      print("  Average training loss: {0:.2f}".format(avg_train_loss))
      print("  Training epcoh took: {:}".format(training_time))
          
      # ========================================
      #               Validation
      # ========================================
      # After the completion of each training epoch, measure our performance on
      # our validation set.

      print("")
      print("Running Validation...")

      t0 = time.time()

      # Put the model in evaluation mode--the dropout layers behave differently
      # during evaluation.
      model.eval()

      # Tracking variables 
      total_eval_accuracy = 0
      total_eval_loss = 0
      nb_eval_steps = 0

      # Evaluate data for one epoch
      for batch in validation_dataloader[i]:
          
          # Unpack this training batch from our dataloader. 
          #
          # As we unpack the batch, we'll also copy each tensor to the GPU using 
          # the `to` method.
          #
          # `batch` contains three pytorch tensors:
          #   [0]: input ids 
          #   [1]: attention masks
          #   [2]: labels 
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)
          
          # Tell pytorch not to bother with constructing the compute graph during
          # the forward pass, since this is only needed for backprop (training).
          with torch.no_grad():        

              # Forward pass, calculate logit predictions.
              # token_type_ids is the same as the "segment ids", which 
              # differentiates sentence 1 and 2 in 2-sentence tasks.
              # The documentation for this `model` function is here: 
              # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
              # Get the "logits" output by the model. The "logits" are the output
              # values prior to applying an activation function like the softmax.
              (loss, logits) = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
              
          # Accumulate the validation loss.
          total_eval_loss += loss.item()

          # Move logits and labels to CPU
          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()

          # Calculate the accuracy for this batch of test sentences, and
          # accumulate it over all batches.
          total_eval_accuracy += flat_accuracy(logits, label_ids)
          

      # Report the final accuracy for this validation run.
      avg_val_accuracy = total_eval_accuracy / len(validation_dataloader[i])
      print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

      # Calculate the average loss over all of the batches.
      avg_val_loss = total_eval_loss / len(validation_dataloader[i])
      
      # Measure how long the validation run took.
      validation_time = format_time(time.time() - t0)
      
      print("  Validation Loss: {0:.2f}".format(avg_val_loss))
      print("  Validation took: {:}".format(validation_time))

      # Record all statistics from this epoch.
      training_stats[i].append(
          {
              'epoch': epoch_i + 1,
              'Training Loss': avg_train_loss,
              'Valid. Loss': avg_val_loss,
              'Valid. Accur.': avg_val_accuracy,
              'Training Time': training_time,
              'Validation Time': validation_time
          }
      )

  print("")
  print("Training complete!")

  print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
32


RuntimeError: ignored